<a href="https://colab.research.google.com/github/vlamen/tue-deeplearning/edit/main/practicals/P5.2_VAE_Anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical 5.2: Anomaly Detection with VAEs

As we have seen, Variational Autoencoders (VAEs) provide a mathematically grounded framework for the unsupervised learning of latent representations. Besides interpreting VAEs as representation learning or generative modelling, we can also see them as performing (approximate) density estimation. VAEs are trained to optimise a lower bound to the (log) likelihood $\log p(X)$ of the data $X$, under the chosen model. So, for any point in data space, we can obtain an estimate of its likelihood under the trained model, by simply computing the loss function when passing this data point through the neural network (note that the loss function is the negative ELBO, so we need to multiply by -1 to obtain a likelihood estimation).

We can use this idea to perform unsupervised anomaly detection. Suppose we are given a dataset that describes some natural distribution (e.g. images of certain clothing items). For new test data, we then wish to detect whether it fits this distribution, or is significantly different (an anomaly). For example, given a dataset of shirts, we want to detect anomalies in a test data set that also contains some images of trousers. Typically, such a situation occurs when we have many examples of one class (e.g. shirts), but very few of others (the anomalies, e.g. trousers).

### Why generative models?
We could describe anomaly detection as binary classification, and train typical supervised methods on data with labels (normal vs. anomaly). However, anomaly detection settings typically have high data imbalance; there are way more normal examples than anomalies. Moreover, anomalies tend to be diverse; there are many ways in which anomalous data can differ from normal data. Supervised methods often don't work well in such cases, and may overfit easily, especially on the anomaly class. Using generative models however, we can ignore the anomaly class completely during training and instead focus on modelling the normal class well. If a generative model can learn to represent normal data well, but is restrictive enough such that it won't be able to represent anomalous data well, it can be much more reliable for anomaly detection.

### Exploring anomaly detection with VAEs.
In this practical, we will perform and evaluate such anomaly detection with VAEs. Given a training data set that consists of instances that we consider "normal", we wish to detect anomalies in a test data set that contains both "normal" (but unseen) examples, as well as other examples which we consider anomalous. The idea is to train a VAE on the training data, such that it learns to represent "normal" data well. We can then compute the ELBO values for the test data, where ideally "normal" examples should obtain higher likelihood values than anomalous examples.

We will use FashionMNIST to simulate the anomaly detection task. We will omit one class from the training data, and consider the remaining 9 classes to be "normal". The goal is then to identify the omitted class in the test data, by comparing the ELBO values obtained from a VAE trained on 9 classes.

# Imports

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision.datasets import FashionMNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import roc_curve, precision_recall_curve, auc

# Data: FashionMNIST with one anomalous class

We will consider the "Trouser" class (with label 1) to be the anomalies, and consider the other 9 classes to be our "normal" data. We'll train on normal data only, but we want to test on both normal and anomalous data to evaluate our anomaly detection framework.

We load the FashionMNIST dataset as usual, but create different DataLoaders for normal and anomalous data.

In [ ]:
dataset_path = '~/datasets'

mnist_transform = transforms.Compose([
        transforms.ToTensor(),
])

train_dataset = FashionMNIST(dataset_path, transform=mnist_transform, train=True, download=True)
test_dataset = FashionMNIST(dataset_path, transform=mnist_transform, train=False, download=True)

In [ ]:
anomaly_label = 1
batch_size = 100
kwargs = {'num_workers': 1, 'pin_memory': True}

# obtain the indices corresponding to normal and anomalous data
anomaly_indices_train = (train_dataset.targets == anomaly_label).nonzero().flatten()
anomaly_indices_test = (test_dataset.targets == anomaly_label).nonzero().flatten()
normal_indices_train = (train_dataset.targets != anomaly_label).nonzero().flatten()
normal_indices_test = (test_dataset.targets != anomaly_label).nonzero().flatten()

# use a SubsetRandomSampler to sample only images from the normal/anomaly dataset, respectively
anomaly_sampler_train = SubsetRandomSampler(anomaly_indices_train)
anomaly_sampler_test = SubsetRandomSampler(anomaly_indices_test)
normal_sampler_train = SubsetRandomSampler(normal_indices_train)
normal_sampler_test = SubsetRandomSampler(normal_indices_test)

# setup DataLoaders
anomaly_loader_train = DataLoader(dataset=train_dataset, batch_size=batch_size, sampler=anomaly_sampler_train, **kwargs)
anomaly_loader_test = DataLoader(dataset=test_dataset, batch_size=batch_size, sampler=anomaly_sampler_test, **kwargs)
normal_loader_train = DataLoader(dataset=train_dataset, batch_size=batch_size, sampler=normal_sampler_train, **kwargs)
normal_loader_test = DataLoader(dataset=test_dataset, batch_size=batch_size, sampler=normal_sampler_test, **kwargs)

Let's quickly plot some images of normal and anomalous images, from both the train and test sets, to check if the split is done correctly.

In [ ]:
def show_images(x, ncols=10):
    """ plots first ncols images in a batch """
    x = x.view(batch_size, 28, 28)

    fig, ax = plt.subplots(1, ncols, figsize=(20, 2))
    for idx in range(ncols):
        ax[idx].imshow(x[idx].cpu().numpy(), cmap="Greys")
        ax[idx].axis('off')
    plt.show()

for loader in [anomaly_loader_train, anomaly_loader_test, normal_loader_train, normal_loader_test]:
    for (x, y) in loader:
        show_images(x)
        break

# Implement and train VAE

## *Practical assignment:*
*Design, implement, and train a VAE **on the normal training data only**. Some hints and ideas:*
* *You can stick to a fully connected architecture as in Tutorial 5.1, or try a convolutional architecture since we're dealing with images here.*
* *Make sure you only train the VAE on normal data. The idea is that the VAE learns to represent (and perform density estimation for) normal data only. Anomalous data can then (hopefully) be detected since the model hasn't been trained for it, and thus will not be able to represent it as well and will result in higher loss values (i.e. lower likelihood estimations).*
* *In Tutorial 5.1 we used a 2-dimensional latent space, mainly since it allows for easy inspection of the latent space. But a higher dimension may work better to learn a good model. If the latent dimension is too high on the other hand, your model may become too flexible, which could actually hurt your anomaly detection performance later on. A balance is needed: ideally your VAE should be flexible enough to represent the normal data well, but not so flexible that it's good at representing anomalous data as well.*


In [ ]:
### INSERT CODE HERE ###

# Evaluation: reconstructions, generate data, ELBO values.

Now let's evaluate the VAE. First, we will qualitatively inspect the VAE itself, by plotting some reconstructions and generating some data (note that we can no longer plot make the 2-dimensional latent space plots from Tutorial 5.1, if we've chosen a latent dimension higher than 2).

Then, we will obtain ELBO values for all data points in both test sets (normal and anomalous), which we can then use to evaluate anomaly detection. Remember that the VAE loss function is in fact based on the **negative** ELBO, and we left out a constant term in the reconstruction loss as well. In Tutorial 5.1 we derived the reconstruction term
\begin{align}
- \log p(\textbf{x}|\textbf{z}) = \sum_{d=1}^D \left( \frac{(x_d-\mu_d)^2}{2\sigma^2} + \frac{1}{2} \log 2\pi\sigma^2 \right).
\end{align}

If we again set $\sigma = \frac{1}{\sqrt{2}}$, we see that the full term should be
\begin{align}
\sum_{d=1}^D \left( (x_d-\mu_d)^2 + \frac{1}{2} \log \pi \right) = \left(\sum_{d=1}^D  (x_d-\mu_d)^2 \right) + \frac{D}{2} \log \pi,
\end{align}
but for our loss function we left out the constant term $\frac{D}{2} \log \pi$. Let's add it back in now to compute correct ELBO values, making sure that we also take the negative values of all the loss terms, since the loss function represents the **negative** ELBO.

Furthermore, we want to obtain ELBO values for each data point individually, whereas the loss function as implemented in Tutorial 5.1 sums up the loss values for a full batch, yielding a single value per batch. We can achieve this by summing only over dim=1 (skipping dim=0, which is the batch dimension).

We can put all this together in the following function:

In [ ]:
def elbo_elementwise(x, x_reconstr, mu, log_sigma):
    reconstr_loss = torch.sum(nn.functional.mse_loss(x_reconstr, x, reduction='none'), dim=1)
    constant_term = x_dim * 0.5 * np.log(np.pi)  # assuming sigma=1/sqrt(2)
    kl_loss = 0.5 * torch.sum(mu.pow(2) + (2*log_sigma).exp() - 2*log_sigma - 1, dim=1)
    elbo = - reconstr_loss - constant_term - kl_loss
    return elbo, reconstr_loss, kl_loss

## *Practical assignment:*

* *Loop through both test sets (normal and anomalous) and use the VAE to obtain reconstructions as well as individual ELBO values for anomaly detection.*
* *Save the ELBO scores to numpy arrays, one for normal test data, one for anomalous test data.*
* *Plot some images and their reconstructions, for both the normal and anomalous test set.*
* *Generate some data by sampling latent variables from the prior, and decoding them into images.*

*Compare the reconstructions for normal data with those for anomalous data. What do you notice? Is this what you'd expect and want?*

*Also inspect the generated images. Do you recognise any trousers? Should you?*

In [ ]:
### INSERT CODE HERE ###

# Anomaly Detection: density plots

We now have two sets of ELBO (= approximate likelihood) scores, one for normal test data and one for anomalous test data. Ideally, we'd want all anomalous data points to have a lower likelihood than all normal data, but this is hard to achieve in practice. Let's first visually inspect how well we did.

## *Practical assignment:*
*Visualise the ELBO scores in a histogram (`plt.hist()`) as well as a density plot (`sns.kdplot` from the `seaborn` library). Use two different colours: **green** for normal data, **red** for anomalous data, and show both normal and anomalous scores in the same plot (use transparency to make the visualisation clearer), i.e. one figure with both histograms, and one figure with both density plots.*

*Based on these plots, do you expect anomaly detection to work well?*

In [ ]:
### INSERT CODE HERE ###

# Anomaly Detection: ROC/PR curves

Given the ELBO (= approximate likelihood) scores, we can choose a threshold and classify all instances with a likelihood below the threshold as anomalies, and all instances with a likelihood above the threshold as "normal". Different thresholds will give different True/False Positive/Negative scores. We can summarise the performance of all possible thresholds in an ROC curve, or a Precision-Recall curve (the latter has been shown to be more suitable for imbalanced datasets, see https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4349800/). In particular, the area under a ROC or Precision-Recall curve can be used as a quantitative measure to evaluate anomaly detection.

## *Practical assignment:*
* *Plot an ROC curve for your results, compute and show the Area Under Curve (AUC) score for quantitative evaluation.*
* *Plot a Precision-Recall curve for your results, compute and show the Area Under Curve (AUC) score for quantitative evaluation.*

*Implementation hints:*
* *See https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/ for more information about ROC and PR curves and their implementation with `sklearn`.*
* *ROC/PR curves are typically used for binary classification. You can define normal as label 0 and anomaly as 1, and rescale the ELBO scores from part (a) to be between 0 and 1 (by using the minimum and maximum values in your ELBO results) to express them as binary classification probabilities.*
    * *Don't forget to "flip" the scores, low (anomalous) ELBO scores should be close to 1, whereas high (normal) ELBO scores should be close to 0.*

In [ ]:
### INSERT CODE HERE ###